In [1]:
import pickle
import os
import pandas as pd
import numpy as np


def load_models(mypath):

    count = 0
    # How many files?
    for path in os.listdir(mypath):
        if os.path.isfile(os.path.join(mypath, path)):
            count += 1

    models = []
    for i in range(0,count):
        with open(mypath+f"m{i}.pkl", "rb") as f:
            models.append(pickle.load(f))
            
    return models, count


def return_movie_Id(df, movie_name):
    mask = df['title'] == movie_name.strip().lower()
    res = df.loc[mask]
    if not res.empty:
        res = res['movieId'].unique().tolist()
    else:
        res = None
    return res
    

def make_ensemble_predictions(models, count, userId, movieId):
    preds = []
    for i in range(count):
        p1 = models[i].predict(userId, movieId).est
        preds.append(p1)
    
   
    preds = np.array(preds)
    votes = preds>2.5
    nvotes = np.count_nonzero(votes)
    if nvotes>5:
        return "Recomended Movie"
    else:
        return "not recommended Movie"

# userId = 12840, movieId = 'as230'  true_rating = 4.5  --> recommended
# userId = 16285, movieId = as2269 true_rating = 2.5  --> not recommended

In [2]:
df = pd.read_csv('../data/clean/all_ratings_with_title.csv', index_col=0)
df.head()

,movieId,userId,score,title
0,as680,1,1.0,the english civil war
1,ns2186,1,4.5,latte and the magic waterstone
2,hs2381,1,5.0,la diosa coronada
3,ns3663,1,5.0,"frankenstein’s monster’s monster, frankenstein"
4,as9500,1,5.0,kept woman


---
### Problem: several movies with same name

---

In [3]:
mask = df['title']=='national treasure'
df.loc[mask]['movieId'].unique()

array(['ds552', 'ns7565', 'hs2756'], dtype=object)

In [4]:
return_movie_Id(df, 'national treasure')

['ds552', 'ns7565', 'hs2756']

In [5]:
return_movie_Id(df, 'nicole')

['as7565']

In [9]:
import pickle
import numpy as np
import gradio as gr

mypath = '../data/models/ensamble1/'
models, count = load_models(mypath)

mypath = '../data/clean/'

# df = pd.read_csv('../data/clean/all_ratings_with_title.csv', index_col=0)

def make_predictions( userId, movieName):
    
    movieId = return_movie_Id(df, movieName)
    print("movieId (inside make predictions)--> ", movieId)
    print("movieName (inside make predictions)--> ", movieName)
    pred = ""
    if movieId!=None:
        if len(movieId)>=1:
            for i in range(len(movieId)):
                pred += "recomendation number "+str(i+1)+" --> "+make_ensemble_predictions(models, count, userId, movieId[i])+"\n"
        
    else:
        pred = "The movie doesnt exist"
    return pred



# userId=12840,score=4.5, movieId=as230

In [ ]:
#Create the input component for Gradio since we are expecting 4 inputs

userId_input = gr.Number(label = "Enter the userId")
# employment_input = gr.Number(label= "Enter Employement Status {1:For Employed, 2: For Unemployed}")
move_input = gr.Textbox(label = "Enter the movie")
# account_input = gr.Number(label = "Enter your account Balance:")
# We create the output
output = gr.Textbox()


app = gr.Interface(fn = make_predictions, inputs=[userId_input, move_input], outputs=output)
app.launch()

userId = 26842	movieId = s7565	score = 1.0  movie=nicole  or movie= national treasure
userId = 25060	movieId =s8013	score = 5.0

In [8]:
df.loc[df['score']<2]

,movieId,userId,score,title
0,as680,1,1.0,the english civil war
24,as7193,1,0.5,the tangle
42,hs1914,2,1.0,into the ashes
43,as9455,2,1.0,europen people
84,ns3279,4,1.0,sunganges
...,...,...,...,...
11024179,as9146,124374,1.0,my kingdom come
11024192,hs282,124375,1.0,i spit on your grave 2
11024218,ns5214,124376,1.0,patton oswalt: annihilation
11024268,ns4842,124380,1.5,november 13: attack on paris


movieId (inside make predictions)-->  ['as680']
